In [ ]:
import zmq
import json
import random
from SimioEnv import SimioPickDontMoveEnv
from gym import spaces
import numpy as np
from gym_helpers import flatten_space_sample
import tensorflow as tf

from IPython.core.debugger import set_trace # https://medium.com/@chrieke/jupyter-tips-and-tricks-994fdddb2057

## Simio Interface Test

### test1

### test2

In [1]:
env = SimioPickDontMoveEnv(log_output=True, log_end_episode_only=False)

print("Num Pickers: ", env.num_pickers)
print("Num AGVs: ", env.num_agvs)
print("Num Warehouse Locations: ", env.num_locations)

print()
print("Action Space:")
print("=============")
print(env.action_space)
print()
print("Observation Space:")
print("==================")
for statevar in env.observation_space:
    print(statevar)

NameError: name 'SimioPickDontMoveEnv' is not defined

In [3]:
print("Picker Observation Space:", len(flatten_space_sample(env.picker_observation_space.sample())))
print("Picker Action Space:", len(flatten_space_sample(env.picker_action_space.sample())))
print("AGV Observation Space:", len(flatten_space_sample(env.agv_observation_space.sample())))
print("AGV Action Space:", len(flatten_space_sample(env.agv_action_space.sample())))

Picker Observation Space: 2
Picker Action Space: 1
AGV Observation Space: 10
AGV Action Space: 1


In [4]:
#state = env.reset()
#for i in range(100000):
#    next_state, reward, done, _ = env.steprandom()
#env.finalize()

## Estimators

In [5]:
from FunctionApproximators import PolicyEstimator, ValueEstimator

In [6]:
tf.reset_default_graph()
global_step = tf.Variable(0, name="global_step", trainable=False)
policy_estimator = PolicyEstimator(env)
value_estimator = ValueEstimator(env)

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please switch to tf.train.get_global_step
Instructions for updating:
Use tf.cast instead.


In [7]:
%load_ext tensorboard.notebook
#%reload_ext tensorboard.notebook
logs_path = "./tensorboard_pickdontmovetest18"
summary_writer = tf.summary.FileWriter(logdir=logs_path, graph=tf.get_default_graph()) # , graph=g
%tensorboard --logdir tensorboard_pickdontmovetest18/

Reusing TensorBoard on port 6006 (pid 20912), started 1:07:12 ago. (Use '!kill 20912' to kill it.)

In [8]:

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())

    sample = 
    value_next = value_estimator.predict(sample)
    action_probs = policy_estimator.predict(sample)
    

In [9]:
value_next

0.0

In [10]:
action_probs

[array([0.07692308, 0.07692308, 0.07692308, 0.07692308, 0.07692308,
        0.07692308, 0.07692308, 0.07692308, 0.07692308, 0.07692308,
        0.07692308, 0.07692308, 0.07692308], dtype=float32),
 array([0.08333334, 0.08333334, 0.08333334, 0.08333334, 0.08333334,
        0.08333334, 0.08333334, 0.08333334, 0.08333334, 0.08333334,
        0.08333334, 0.08333334], dtype=float32),
 array([0.08333334, 0.08333334, 0.08333334, 0.08333334, 0.08333334,
        0.08333334, 0.08333334, 0.08333334, 0.08333334, 0.08333334,
        0.08333334, 0.08333334], dtype=float32)]

In [17]:
chosen_actions = []
for prob in action_probs:
    action = np.random.choice(np.arange(len(prob)), p=prob)
    chosen_actions.append(action)

ca = np.array([np.random.choice(np.arange(len(prob)), p=prob) for prob in action_probs])
print(chosen_actions)
print(np.array(chosen_actions))
print(ca)

[4, 4, 9]
[4 4 9]
[9 6 3]


In [12]:
sample

array([[ 1,  4,  1,  3, 64, 69, 84, 53, 60, 64,  6, 19,  2,  8, 68, 92,
        84, 14, 13, 38, 57, 11]], dtype=int64)

In [22]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    # sess.run(tf.initialize_all_variables())
    
    reward = 0.1
    discount_factor = 1.0

    td_target = reward + discount_factor * value_next
    td_error = td_target - value_next 

    # Update the value estimator
    value_loss = value_estimator.update(sample, td_target)
    print(value_loss)
    
    # Update the policy estimator
    # using the td error as our advantage estimate
    policy_loss = policy_estimator.update(sample, td_error, chosen_actions)
    print(policy_loss)
    print(chosen_actions)
    print([int(a) for a in chosen_actions])

0.010000001
[[<tf.Operation 'policy_estimator/Adam' type=AssignAdd>, <tf.Tensor 'policy_estimator/mul:0' shape=<unknown> dtype=float32>], [<tf.Operation 'policy_estimator/Adam_1' type=AssignAdd>, <tf.Tensor 'policy_estimator/mul_1:0' shape=<unknown> dtype=float32>], [<tf.Operation 'policy_estimator/Adam_2' type=AssignAdd>, <tf.Tensor 'policy_estimator/mul_2:0' shape=<unknown> dtype=float32>]]
[0.25649494, 0.24849068, 0.24849068]
[4, 4, 9]
[4, 4, 9]


In [37]:
len(chosen_actions) == len(env.action_space.nvec)

True

In [ ]:
%debug

> <ipython-input-38-f4c406caa424>(1)<module>()
----> 1 env.action_space[0]

